# Import libraries

In [5]:
# Remove the # and run or remove the ! and run it in the console.
#!conda install -c anaconda beautifulsoup4 --y
#!conda install -c conda-forge folium

In [116]:
import requests
import pandas as pd
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import json # library to handle JSON files

from lxml import etree
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


I'm using <code>beautifulsoup</code> to parse HTML elements. The problem is that I didn't read the documentation that came with the "My Submission" so the code might be different. I won't be using <code>geocoder</code> since it's very slow.

# Scrape and Store data

In [117]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')
postal_codes = soup.find('table',{'class':'wikitable sortable'})

df = pd.DataFrame()
for e in enumerate(postal_codes.find_all('tr')):
    if (e[0] > 0):
        content = e[1].text.split('\n');
        
        content.remove('')
        content.remove('')
        
         # Borough should 'Not assigned'
        if (content[1] != 'Not assigned'):
            # Unassigned Neighborhood = Borough
            if (content[2] == 'Not assigned'): content[2] = content[1]
           
            # Entry
            df = df.append({'PostalCode'  : content[0],
                            'Borough'     : content[1],
                            'Neighborhood': content[2],}, 
                           
                           ignore_index=True)

df = df[['PostalCode','Borough','Neighborhood']]
df


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


This is the first time i used the <code>enumerate(...)</code> method, I don't really want to make variables I don't plan to use that often so it's convenient and makes my code a little bit cleaner. The <code># Entry</code> part sucks but I'm not that experienced to make a cleaner solution.

# Inserting Coordinates

In [118]:
# Store coordinates by PostalCode
df2 = pd.DataFrame()
with open('csv/Geospatial_Coordinates.csv') as file:
    while True:
        line = file.readline().strip('\n')
        content = line.split(',')
        
        if (len(content) == 3 and len(content[0]) == 3):
            
            df2 = df2.append({'PostalCode' : content[0],
                                'Latitude' : content[1],
                                'Longitude': content[2],}, ignore_index=True)
        if (line == ''): break
                         
df = df.merge(df2, how='left', left_on='PostalCode', right_on='PostalCode')

# Map the neighborhoods to the... Map

In [120]:
# create map of New York using latitude and longitude values
# The df from Out[33] shows that all of them are 43, -79 -ish
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=15)

# add markers to map
for latitude, longitude, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Define Foursquare Credentials and Version

In [121]:
# It really doesn't matter if other people use this
CLIENT_ID = '3F25XT5NW0405TVRIPESYQAOOIDYICNMWRX0FFFY4WVMVCPJ' # your Foursquare ID
CLIENT_SECRET = 'VILP5ZFM1UFTPOQLUSDM0T0MBKLWFIQF4W1RMI0SWBEYECLW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3F25XT5NW0405TVRIPESYQAOOIDYICNMWRX0FFFY4WVMVCPJ
CLIENT_SECRET:VILP5ZFM1UFTPOQLUSDM0T0MBKLWFIQF4W1RMI0SWBEYECLW


# Explore the Neighborhoods in Toronto with Foursquare

In [122]:
# If you want a quicker evaluation and a smaller dataset, feel free to lower the LIMIT
# If you want more or less coverage, feel free to play with RADIUS.

LIMIT = 100
RADIUS = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto 

In [123]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

----Adelaide----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.04


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4                Yoga Studio  0.00


----Agincourt North----
                        venue  freq
0                        Park   0.5
1                  Playground   0.5
2                 Yoga Studio   0.0
3                 Men's Store   0.0
4  Modern European Restaurant   0.0


----Albion Gardens----
                  venue  freq
0              Pharmacy  0.11
1  Fast Food Restaurant  0.11
2           Pizza Place  0.11
3   Japanese Restaurant  0.11
4         Grocery Store  0.11


----Alderwood----
                venue  freq
0         Pizza Place   0.2
1        Skating Rink   0.1
2      Sandwich Place   0.1
3  Athletics & Sports   

4         Grocery Store  0.11


----Keelesdale----
                   venue  freq
0             Restaurant   0.2
1         Discount Store   0.2
2  Check Cashing Service   0.2
3         Sandwich Place   0.2
4            Coffee Shop   0.2


----Kennedy Park----
               venue  freq
0     Discount Store  0.29
1  Convenience Store  0.14
2        Bus Station  0.14
3   Department Store  0.14
4        Coffee Shop  0.14


----Kensington Market----
                   venue  freq
0                   Café  0.08
1                    Bar  0.05
2     Chinese Restaurant  0.04
3    Dumpling Restaurant  0.04
4  Vietnamese Restaurant  0.04


----King----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.04


----King and Spadina----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4   Harbor / Marina  0.06


----King's Mil

         venue  freq
0  Coffee Shop  0.09
1         Café  0.04
2   Restaurant  0.04
3        Hotel  0.03
4       Bakery  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2  American Restaurant  0.05
3   Italian Restaurant  0.05
4               Bakery  0.05


----Sullivan----
                venue  freq
0         Pizza Place  0.17
1            Pharmacy  0.17
2                Bank  0.08
3        Noodle House  0.08
4  Italian Restaurant  0.08


----Summerhill East----
           venue  freq
0     Restaurant  0.25
1           Park  0.25
2   Tennis Court  0.25
3     Playground  0.25
4  Metro Station  0.00


----Summerhill West----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2           Bagel Shop  0.07
3  American Restaurant  0.07
4           Restaurant  0.07


----Sunnylea----
                        venue  freq
0              Breakfast Spot   0.5
1              Baseball Field   

# Clustering

What i did is basically a copy-paste from the lab. I don't think there's anything unique to be done...

In [124]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


# Map Clusters

In [128]:

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining Clusters

In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Downtown Toronto,0.0,Coffee Shop,Park,Pub,Bakery,Café,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Italian Restaurant
3,North York,0.0,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Furniture / Home Store,Arts & Crafts Store,Gift Shop,Carpet Store,Boutique,Accessories Store
4,North York,0.0,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Furniture / Home Store,Arts & Crafts Store,Gift Shop,Carpet Store,Boutique,Accessories Store
5,Queen's Park,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Sandwich Place,Bar,Portuguese Restaurant,Café,Yoga Studio
6,Queen's Park,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Sandwich Place,Bar,Portuguese Restaurant,Café,Yoga Studio
9,North York,0.0,Café,Basketball Court,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
10,East York,0.0,Fast Food Restaurant,Pizza Place,Gastropub,Pharmacy,Bus Line,Breakfast Spot,Intersection,Bank,Athletics & Sports,Gym / Fitness Center
11,East York,0.0,Fast Food Restaurant,Pizza Place,Gastropub,Pharmacy,Bus Line,Breakfast Spot,Intersection,Bank,Athletics & Sports,Gym / Fitness Center
12,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Bakery,Fast Food Restaurant,Middle Eastern Restaurant,Café,Diner,Italian Restaurant,Ramen Restaurant
